# side model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import SIDE

df_features = pd.read_parquet("data/features.parquet")

features = df_features[SIDE]
label = np.load("data/side_label.npy")
len_gap = len(label) - len(df_features)
label = label[len_gap:]

print(features.shape)
print(label.shape)

mask = features.index < date_to_timestamp("2025-01-01")
features_masked = features[mask]
label_masked = label[mask]

print(features_masked.shape)
print(label_masked.shape)
print(np.unique(label_masked, return_counts=True))

features.isna().sum().sort_values(ascending=False)

(13405, 487)
(13405,)
(12523, 487)
(12523,)
(array([0, 1]), array([6992, 5531]))


williams_r                        0
evenbetter_sinewave_short_lag8    0
amihud_lambda_lag18               0
swamicharts_stochastic_17         0
highpass_bp_lag19                 0
                                 ..
swamicharts_stochastic_37         0
aroon_diff_lag4                   0
conv_0                            0
swamicharts_stochastic_15         0
chaiken_money_flow_lag17          0
Length: 487, dtype: int64

In [2]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "metric": "auc",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(features_masked, label_masked)
res = lgb.cv(params, dtrain, num_boost_round=100, nfold=5, stratified=True)
res.keys()

dict_keys(['valid auc-mean', 'valid auc-stdv'])

In [3]:
res["valid auc-mean"][-1]

0.9180293366803529

In [4]:
import lightgbm as lgb
import optuna


def objective(trial):
    METRIC = "auc"

    params = {
        "objective": "binary",
        "metric": METRIC,
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(features, label)
    # dtest = lgb.Dataset(side_features_test, side_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
        metrics=METRIC,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-06-08 11:38:38,798] A new study created in memory with name: no-name-50cfb4e0-8e0b-44e5-b353-b4a0e1d6be83


[I 2025-06-08 11:39:13,721] Trial 0 finished with value: 0.8974541949015735 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 190, 'max_depth': 855, 'min_gain_to_split': 0.46842096407303535, 'min_data_in_leaf': 44, 'lambda_l1': 93.08507564957453, 'lambda_l2': 69.89099645918385, 'num_boost_round': 966}. Best is trial 0 with value: 0.8974541949015735.
[I 2025-06-08 11:39:22,004] Trial 1 finished with value: 0.9134164417809906 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 148, 'max_depth': 963, 'min_gain_to_split': 0.6793015143880743, 'min_data_in_leaf': 29, 'lambda_l1': 2.573631242371378, 'lambda_l2': 97.63221651265196, 'num_boost_round': 707}. Best is trial 1 with value: 0.9134164417809906.
[I 2025-06-08 11:39:35,232] Trial 2 finished with value: 0.8998443071212984 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 166, 'max_depth': 422, 'min_gain_to_

In [5]:
print("side model long label: ")
study.best_params

side model long label: 


{'is_unbalance': True,
 'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 62,
 'max_depth': 294,
 'min_gain_to_split': 0.3876711100653668,
 'min_data_in_leaf': 246,
 'lambda_l1': 0.3366923231961449,
 'lambda_l2': 47.334403917307704,
 'num_boost_round': 586}

In [6]:
import lightgbm as lgb  # noqa

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

dtrain = lgb.Dataset(features_masked, label_masked)
side_model = lgb.train(params, dtrain)
side_model_prod = lgb.train(params, lgb.Dataset(features, label))

In [7]:
import plotly.express as px

test_features = features[features.index > date_to_timestamp("2025-01-01")]
res = side_model.predict(test_features)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
side_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_side.txt")
side_model_prod.save_model(
    "strategies/BinanceBtcEntropyBarV1/model/model_side_prod.txt"
)

# meta model

In [1]:
import numpy as np
import pandas as pd

from jesse.helpers import date_to_timestamp

from strategies.BinanceBtcEntropyBarV1.config import (
    META_ALL,
    SIDE,
    get_side_model,
)

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = get_side_model(False)

side_model_res = side_model.predict(df_features[SIDE])

df_features["model"] = side_model_res

meta_features = df_features[META_ALL]
print(meta_features.shape)
print(meta_label.shape)

mask = meta_features.index < date_to_timestamp("2025-01-01")
meta_features_masked = meta_features[mask]
meta_label_masked = meta_label[mask]

print(meta_features_masked.shape)
print(meta_label_masked.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(13405, 383)
(13405,)
(12523, 383)
(12523,)


corwin_schultz_estimator            0
ac_28                               0
dft_dom_cycle_ddt_lag18             0
corwin_schultz_estimator_dt_lag1    0
adaptive_stochastic                 0
                                   ..
evenbetter_sinewave_long_lag17      0
evenbetter_sinewave_short_dt        0
swamicharts_stochastic_23           0
acc_swing_index_dt_lag3             0
model                               0
Length: 383, dtype: int64

In [2]:
import lightgbm as lgb
from sklearn.metrics import (
    f1_score,  # noqa
    fbeta_score,  # noqa
)

METRIC = "f1"


def eval_metric(preds, eval_dataset):
    metric_name = METRIC
    y_true = eval_dataset.get_label()
    value = f1_score(y_true, preds > 0.5, average="macro")
    higher_better = True
    return metric_name, value, higher_better


params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    "is_unbalance": True,
    "extra_trees": False,
    "num_leaves": 100,
    "max_depth": 20,
    "min_gain_to_split": 1e-8,
    "min_data_in_leaf": 20,
    "lambda_l1": 1e-4,
    "lambda_l2": 1e-4,
}
dtrain = lgb.Dataset(meta_features, meta_label)
res = lgb.cv(
    params, dtrain, num_boost_round=100, nfold=5, stratified=True, feval=eval_metric
)
res.keys()

dict_keys(['valid binary_logloss-mean', 'valid binary_logloss-stdv', 'valid f1-mean', 'valid f1-stdv'])

In [3]:
import optuna


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features, meta_label)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model_res = lgb.cv(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 1000),
        stratified=True,
        feval=eval_metric,
    )
    return model_res[f"valid {METRIC}-mean"][-1]


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-06-08 13:51:50,986] A new study created in memory with name: no-name-f4d07824-73df-4b55-b58e-5e13601dd294
[I 2025-06-08 13:51:58,741] Trial 0 finished with value: 0.8040286404799428 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 497, 'max_depth': 326, 'min_gain_to_split': 0.4240174454504108, 'min_data_in_leaf': 245, 'lambda_l1': 75.82930538605103, 'lambda_l2': 74.84875410561239, 'num_boost_round': 982}. Best is trial 0 with value: 0.8040286404799428.
[I 2025-06-08 13:52:15,872] Trial 1 finished with value: 0.7946656232883063 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 484, 'max_depth': 736, 'min_gain_to_split': 0.7578933118027624, 'min_data_in_leaf': 179, 'lambda_l1': 68.12515847171913, 'lambda_l2': 61.954466238083754, 'num_boost_round': 578}. Best is trial 0 with value: 0.8040286404799428.
[I 2025-06-08 13:52:35,838] Trial 2 finished with value: 0.7723932329652335 and parameters

In [4]:
print("meta model: ")
study.best_params

meta model: 


{'is_unbalance': False,
 'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 377,
 'max_depth': 133,
 'min_gain_to_split': 0.07755900215445423,
 'min_data_in_leaf': 186,
 'lambda_l1': 2.724682203790961,
 'lambda_l2': 88.17441637056821,
 'num_boost_round': 970}

In [5]:
import lightgbm as lgb

params = {
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_masked, meta_label_masked),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [6]:
import plotly.express as px

res = model.predict(
    meta_features[meta_features.index > date_to_timestamp("2025-01-01")]
)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [7]:
model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta.txt")
prod_model.save_model("strategies/BinanceBtcEntropyBarV1/model/model_meta_prod.txt")